# 03. 서포트 벡터 머신
> 커널기반의 서포트 벡터머신 알고리즘을 배워보고 실습을 진행해봅시다.

- toc: true 
- badges: true
- comments: true
- categories: [Day 4]
- permalink: /svm
- exec: colab

<br><br>

### 1. 선을 조금 더 잘 그어보자

![](https://i0.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm01.png?w=1372)

로지스틱 회귀 알고리즘에서 우리는 Sigmoid 함수와 Cross Entropy Loss를 활용해서 데이터를 분류하는 선(Decision Boundary)을 찾는 방법에 대해서 배웠습니다. 
<br><br>

![](https://i0.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm02.png?resize=1536%2C1278)

만약 데이터가 3차원이라면 위처럼 평면(Hyperplane)으로 분리를 해내겠죠? 이러한 내용은 로지스틱 회귀와 미적분 시간에 모두 다루었습니다. 
<br><br>

![](https://i1.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm03.png?w=1280)

그런데 이러한 선이나 면은 위 처럼 매우 다양하게 그릴 수 있습니다. 그렇다면 **과연 어떻게 분리해내는게 좋은 방법일까요?** 위 그림의 C나 B를 보면 파란색쪽에 너무 아슬아슬하게 붙어있습니다. **만약 값이 조금 큰 파란색 샘플이 들어오면 오답으로 예측할 것입니다.** 그에 비해 F는 어느정도는 여유가 있죠
<br><br>

![](https://i0.wp.com/hleecaster.com/wp-content/uploads/2020/01/svm04.png?w=1280)

서포트 벡터 머신은 로지스틱 회귀에서 한 단계 더 발전해서 분류하기 좋은, 최적의 선을 찾아냅니다. 분류를 위한 **선이 기본적으로 근접한 데이터 샘플들과 위치상 멀어야 한다는 것**을 전제로 하고 선을 긋는 방법에 대해 학습합니다. 
<br><br>

이렇게 가장 근접한 데이터 샘플들과의 거리를 **마진(Margin)** 이라고 합니다. 위 그림에서 파란색 영역을 의미합니다. 저렇게 **마진(Margin)** 이 넓어지면 데이터를 더욱 잘 맞출 확률이 높아집니다. 또한, 마진과 맞닿아 있는 가장 근접한 샘플들을 **서포트 벡터*(Support Vector)** 라고 합니다. 이 서포트 벡터들의 위치를 이용해 마진(Margin)을 계산하고, 이 마진을 최대로 벌리도록 학습합니다. 
<br><br>

![](https://github.com/gusdnd852/bigdata-lecture/blob/master/_notebooks/img/Day4/02.png?raw=True)

서포트벡터 머신은 굉장히 수학적으로 탄탄하게 정의 된 모델입니다. 공부하기에는 상당히 어려운 난이도에 속하기 때문에 수학적인 내용은 여기에서 생략하도록 하겠습니다. 관심있으신 분들은 '목적식과 제약식', '라그랑지안 승수법', 'KKT 조건', 'Wolf 쌍대성'등을 공부하시고 나서 SVM을 공부하시면 이해에 수월하실 것입니다. 만약 개인적으로 궁금하시면 저한테 물어보시면 알려드리겠습니다. (웬만한 모델은 수학적으로 다 다뤄드릴려고 했는데 SVM 강의하려면 최소 3일은 SVM만 해야합니다)

### 2. 커널 함수 정의